# PCA on Iris

In [2]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_csv('../data/processed/iris_clean.csv')
raw = pd.read_csv('../data/raw/iris.csv')
df.head()
feature_cols=['sepal.length','sepal.width','petal.length','petal.width']
X=df[feature_cols]
y=raw['variety']
X.head()
y.head()

0    Setosa
1    Setosa
2    Setosa
3    Setosa
4    Setosa
Name: variety, dtype: object

## Covariance matrix

In [11]:
cov_matrix=X.T@X/(X.shape[0]-1)
print(cov_matrix)
cov_matrixNumpy=np.cov(X,rowvar=False)
print(cov_matrixNumpy)

              sepal.length  sepal.width  petal.length  petal.width
sepal.length      1.000000    -0.117570      0.871754     0.817941
sepal.width      -0.117570     1.000000     -0.428440    -0.366126
petal.length      0.871754    -0.428440      1.000000     0.962865
petal.width       0.817941    -0.366126      0.962865     1.000000
[[ 1.         -0.11756978  0.87175378  0.81794113]
 [-0.11756978  1.         -0.4284401  -0.36612593]
 [ 0.87175378 -0.4284401   1.          0.96286543]
 [ 0.81794113 -0.36612593  0.96286543  1.        ]]


## Computing eigenvalues and eigenvectors

In [25]:
# I=np.eye(cov_matrix.shape[0])
# print(I)
# TransformMatrix=cov_matrix-I
# print(TransformMatrix)
##########################
eigvals,eigvecs=np.linalg.eig(cov_matrix)

print(f'eigen values are:{eigvals},\neigenvectors are :\n{eigvecs}')

pairs=[(eigvals[i],eigvecs[:,i])for i in range(len(eigvals))]
for idx, (λ, v) in enumerate(pairs, start=1):
    print(f"Mode {idx}: λ = {λ:.4f},  v = {v}")
print('\n')
pairs_sort=sorted(pairs,key=lambda pair:pair[0],reverse=True)
for idx, (λ, v) in enumerate(pairs_sort, start=1):
    print(f"Mode {idx}: λ = {λ:.4f},  v = {v}")

eigen values are:[2.91849782 0.91403047 0.14675688 0.02071484],
eigenvectors are :
[[ 0.52106591 -0.37741762 -0.71956635  0.26128628]
 [-0.26934744 -0.92329566  0.24438178 -0.12350962]
 [ 0.5804131  -0.02449161  0.14212637 -0.80144925]
 [ 0.56485654 -0.06694199  0.63427274  0.52359713]]
Mode 1: λ = 2.9185,  v = [ 0.52106591 -0.26934744  0.5804131   0.56485654]
Mode 2: λ = 0.9140,  v = [-0.37741762 -0.92329566 -0.02449161 -0.06694199]
Mode 3: λ = 0.1468,  v = [-0.71956635  0.24438178  0.14212637  0.63427274]
Mode 4: λ = 0.0207,  v = [ 0.26128628 -0.12350962 -0.80144925  0.52359713]


Mode 1: λ = 2.9185,  v = [ 0.52106591 -0.26934744  0.5804131   0.56485654]
Mode 2: λ = 0.9140,  v = [-0.37741762 -0.92329566 -0.02449161 -0.06694199]
Mode 3: λ = 0.1468,  v = [-0.71956635  0.24438178  0.14212637  0.63427274]
Mode 4: λ = 0.0207,  v = [ 0.26128628 -0.12350962 -0.80144925  0.52359713]


## computing explained-Variance Ratio

In [28]:
total_var=np.sum(eigvals)

evr=eigvals/total_var
print(evr)

cum_evr=np.cumsum(evr)
print(cum_evr)

[0.72962445 0.22850762 0.03668922 0.00517871]
[0.72962445 0.95813207 0.99482129 1.        ]
